In [13]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
info = dict()
info['iter_time'] = []
a = 1
info['iter_time'].append(a)
b = 4
info['iter_time'].append(b)
try:    x = parameter['x0']
except: x = np.zeros(10)
x = range(0,10)
x = np.array([1,2,3])
y = np.kron(np.ones((5,1)), x)
tmp = np.array([[4,5],[6,7]])
y[1:3,1:3] = tmp.T
a =np.abs(np.array([1,2])-np.array([2,1]))
a

array([1, 1])

In [39]:
def SAGA(fx, gradf, prox, parameter):
    # Parameter setting
    n            = parameter['n']                   # Number of observations
    d            = parameter['d']                   # Decision variable dimension
    epoch_max    = parameter['epoch_max']           # Max epoch number
    gamma        = parameter['gamma']               # Learning rate
    tol          = parameter['tol']                 # convergence tolerance
    try:    
        x        = parameter['x0']                  # Initial condition
    except: 
        x        = np.zeros(d)
    try:
        use_perm = parameter['perm']                # if true use permutation; otherwise use randomization
    except:
        use_perm = True
    # Output initialization    
    info              = dict()
    info['epoch_max'] = []
    info['fx']        = []
    # Initialization
    gx       = gdraf(x)
    g_phi    = np.kron(np.ones((n,1)), gx)
    g_phi_av = gx
    flag     = True
    for epoch in range(epoch_max):        
        if not flag: break
        perm = np.random.permutation(n)
        for j in range(n):
            t = time.time()
            if epoch == 0:
                i = j
            else:
                if use_perm:
                    if epoch % 2 == 0:
                        i = j 
                    else:
                        i = perm[j]
                else:
                    i = np.random.randint(0, n)
            # Update the next iteration
            gx         = gradf(x)
            w_next     = x - gamma * (gx - g_phi[i,:] + g_phi_av)
            g_phi_av   = g_phi_av - g_phi[i,:] / n + gx / n
            g_phi[i,:] = gx             
            x_next     = prox(w_next, gamma)
            # Save information
            info['iter_time'].append = time.time() - t
            info['fx'].append        = fx(x) 
            # Check optimality condition
            if np.abs(fx(x_next)-fx(x)) <= tol:
                flag = False
                break
            # Prepare the next iteration
            x = x_next             
    info['epoch'] = epoch    
    return (x, info)

In [40]:
def SAG(fx, gradf, prox, parameter):
    # Parameter setting
    n            = parameter['n']                   # Number of observations
    d            = parameter['d']                   # Decision variable dimension
    epoch_max    = parameter['epoch_max']           # Max epoch number
    gamma        = parameter['gamma']               # Learning rate
    tol          = parameter['tol']                 # convergence tolerance
    try:    
        x        = parameter['x0']                  # Initial condition
    except: 
        x        = np.zeros(d)
    try:
        use_perm = parameter['perm']                # if true use permutation; otherwise use randomization
    except:
        use_perm = True
    # Output initialization    
    info              = dict()
    info['epoch_max'] = []
    info['fx']        = []
    # Initialization
    gx        = gdraf(x)
    g_phi     = np.kron(np.ones((n,1)), gx)
    g_phi_sum = n * gx
    flag      = True
    for epoch in range(epoch_max):        
        if not flag: break
        perm = np.random.permutation(n)
        for j in range(n):
            t = time.time()
            if epoch == 0:
                i = j
            else:
                if use_perm:
                    if epoch % 2 == 0:
                        i = j 
                    else:
                        i = perm[j]
                else:
                    i = np.random.randint(0, n)
            # Update the next iteration
            gx         = gradf(x)
            w_next     = x - gamma * (gx - g_phi[i,:] + g_phi_sum) / n
            g_phi_sum  = g_phi_sum - g_phi[i,:] + gx
            g_phi[i,:] = gx             
            x_next     = prox(w_next, gamma)
            # Save information
            info['iter_time'].append = time.time() - t
            info['fx'].append        = fx(x) 
            # Check optimality condition
            if np.abs(fx(x_next)-fx(x)) <= tol:
                flag = False
                break
            # Prepare the next iteration
            x = x_next             
    info['epoch'] = epoch    
    return (x, info)